In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import os

In [2]:
train_df = pd.read_csv('./archive/train.csv')
test_df = pd.read_csv('./archive/test.csv')
train_images = os.getcwd() + '\\archive\\train_images\\' + train_df.iloc[:, 0].values
test_images = os.getcwd() + '\\archive\\test_images\\' + test_df.iloc[:, 0].values

In [3]:
train_images[63997]

'E:\\DL\\CNN\\Multi_Digit_Mnist\\archive\\train_images\\999_98.png'

In [4]:
train_df

,Image,first_num,second_num
0,0_00.png,0,0
1,100_00.png,0,0
2,101_00.png,0,0
3,102_00.png,0,0
4,103_00.png,0,0
...,...,...,...
63995,997_98.png,9,8
63996,998_98.png,9,8
63997,999_98.png,9,8
63998,99_98.png,9,8


In [5]:
train_lables = train_df.iloc[:,1:].values
test_lables = test_df.iloc[:,1:].values

In [6]:
train_lables[63997]

array([9, 8], dtype=int64)

In [7]:
def read_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=1, dtype=tf.float32)
    
    # this 3 lines can be removed in tf version >=2.3.0
    image.set_shape((64,64,1))
    label[0].set_shape([])
    label[1].set_shape([])
    
    labels = {'first_num': label[0], 'second_num': label[1]}
    return image, labels

In [8]:
batch_size = 64
weight_decay = 0.001
learning_rate = 0.001

In [9]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [10]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_images,train_lables))

In [11]:
train_dataset = (train_dataset.shuffle(buffer_size=len(train_lables)).map(read_image)
                 .batch(batch_size=batch_size).prefetch(buffer_size=AUTOTUNE))

In [12]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_lables))

In [13]:
test_dataset = (test_dataset.map(read_image)
                 .batch(batch_size=batch_size).prefetch(buffer_size=AUTOTUNE))

In [14]:
inputs = keras.Input(shape=(64,64,1))
x = layers.Conv2D(32, 3, padding='same', kernel_regularizer = keras.regularizers.l2(weight_decay))(inputs)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.Conv2D(64, 3, kernel_regularizer = keras.regularizers.l2(weight_decay))(x)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(64, 3, activation='relu', kernel_regularizer = keras.regularizers.l2(weight_decay))(x)
x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.MaxPool2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(64, activation='relu')(x)
output1 = layers.Dense(10, name='first_num', activation='softmax')(x)
output2 = layers.Dense(10, name='second_num', activation='softmax')(x)

model = keras.Model(inputs = inputs, outputs = [output1, output2])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), 
              loss=[keras.losses.SparseCategoricalCrossentropy(),
                    keras.losses.SparseCategoricalCrossentropy()],
              metrics=['accuracy']
             )

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 64, 32)   320         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 64, 64, 32)   128         conv2d[0][0]                     
__________________________________________________________________________________________________
tf.nn.relu (TFOpLambda)         (None, 64, 64, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [17]:
model.fit(train_dataset, epochs=10)

Epoch 1/10
1000/1000 [==============================] - 47s 47ms/step - loss: 0.5263 - first_num_loss: 0.2293 - second_num_loss: 0.2256 - first_num_accuracy: 0.9256 - second_num_accuracy: 0.9273
Epoch 2/10
1000/1000 [==============================] - 44s 44ms/step - loss: 0.2556 - first_num_loss: 0.1027 - second_num_loss: 0.0996 - first_num_accuracy: 0.9678 - second_num_accuracy: 0.9680
Epoch 3/10
1000/1000 [==============================] - 46s 46ms/step - loss: 0.1863 - first_num_loss: 0.0709 - second_num_loss: 0.0706 - first_num_accuracy: 0.9775 - second_num_accuracy: 0.9776
Epoch 4/10
1000/1000 [==============================] - 47s 46ms/step - loss: 0.1603 - first_num_loss: 0.0592 - second_num_loss: 0.0599 - first_num_accuracy: 0.9808 - second_num_accuracy: 0.9802
Epoch 5/10
1000/1000 [==============================] - 47s 47ms/step - loss: 0.1334 - first_num_loss: 0.0494 - second_num_loss: 0.0452 - first_num_accuracy: 0.9846 - second_num_accuracy: 0.9848
Epoch 6/10
1000/1000 [===

In [18]:
model.evaluate(test_dataset)

313/313 [==============================] - 65s 206ms/step - loss: 1.0636 - first_num_loss: 0.4638 - second_num_loss: 0.5727 - first_num_accuracy: 0.8785 - second_num_accuracy: 0.8561


[1.0635658502578735,
 0.4638114869594574,
 0.5727300047874451,
 0.8784999847412109,
 0.8561499714851379]

In [19]:
# model.save('Multi_Digit_Mnist.h5')